In [6]:
import warnings

# The workhorses
import numpy as np
import scipy.integrate
import scipy.optimize

In [7]:
def count_connections (filename):
    with open(filename,"r") as f:
        connections={}
        for line in f:
            if line.startswith("#") or line.startswith("Source"):
                continue
            connections.setdefault(line.split()[0],[0,0])
            connections.setdefault(line.split()[1],[0,0])

            connections[line.split()[0]][1]+=1
            connections[line.split()[1]][0]+=1

    return connections 

In [8]:
def build_adj_list(filename, gene_list):
    with open(filename,"r") as f:
        adj_matrix=np.zeros((len(gene_list),len(gene_list)))
        for line in f:
            if line.startswith("#") or line.startswith("Source"):
                continue
            
            source=gene_list.index(line.split()[0])
            target=gene_list.index(line.split()[1])
    
            if line.split()[2] == "+":
                adj_matrix[target][source]=1 
            elif line.split()[2] == "-":
                adj_matrix[target][source]=-1 
                
    return adj_matrix



# Define right hand side of ODEs
def update_gene(weight, regulator, gene, n, K, decay):
    if weight == 0:
        return 0
    elif weight > 0:
        return ((weight * regulator ** n ) / (K + regulator ** n)) - (decay * gene)
    elif weight <0:
        return (weight / (regulator / K) ** n) -(decay * gene) 

                
def dx_dt(x, t, matrix_betas, gene_names, n, K,decay):
    x_updated= [];
    for i, gene in enumerate(gene_names):
        x_updated.append(x[i]+ (sum(update_gene(weight, x[j], x[i], n, K, decay[i]) for j, weight in enumerate(matrix_betas[i]))))
              
    return np.array(x_updated)


In [9]:
#Set up gene list
gene_list=[key for key in count_connections("pruned_net.csv").keys()]

#build matrix
matrix=build_adj_list("pruned_net.csv", gene_list)

# Initial conditions ##MISMO ORDEN QUE GENE_LIST
x0=np.ones(len(gene_list))

# Time points
t = np.linspace(0, 30, 1000)

# Choose parameters
K = 10.0
n = 3
decay=np.ones(len(gene_list))
# Solve it!
x = scipy.integrate.odeint(dx_dt, x0, t, args=(matrix, gene_list, n, K,decay))
print(x[:10])

[[  1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000]
 [  7.11125545e-008  -3.42341302e-010   8.65107784e-001   8.31977029e-001
    1.00001225e+000   8.65107814e-001   1.00001225e+000   1.00001225e+000
    1.00001225e+000   9.99887649e-001   9.99887357e-001   8.65245996e-001
    1.00001015e+000]
 [  2.42188757e-263   1.09902218e-259   3.02407524e+209   3.65830310e-263
    1.17088649e+195   2.02956702e-263   4.06149028e-263   1.23483696e+169
    2.72272671e-263   4.92776730e+231   4.72952136e+233   1.69388749e+166
    3.29486944e-318]
 [  4.10988507e-319   2.19726131e-316   2.19726012e-316   2.19726289e-316
    5.51718906e-313   2.47032823e-323   4.94065646e-324   2.19724985e-316
    9.33678148e-313   5.51718906e-313   0.00000000e+000   0.00000000e+000
    0.00000000e+000]
 [  0.00000000e+000   5.5171

/home/carmen/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [19]:
print(dx_dt(x0,t,matrix,gene_list,n,K,decay))
print(matrix.T)
print(gene_list)

[ -3.00200000e+03  -2.00281818e+03  -1.00090909e+03  -1.00090909e+03
   9.09090909e-02  -1.00090909e+03   9.09090909e-02   9.09090909e-02
   9.09090909e-02  -8.18181818e-01  -8.18181818e-01  -1.00000000e+03
   9.09090909e-02]
[[ 0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]]
['rok', 'comk', 'abrb', 'sigh', 'siga', 'resd', 'sigd', 'm

In [11]:
import networkx as nx

In [13]:
file="pruned_net.csv"
net=nx.read_edgelist(file, comments='#', delimiter="\t", nodetype =str,  data=(('mode',str),), create_using=nx.DiGraph())
net.edges(data=True)

[('rok', 'comk', {'mode': '-'}),
 ('comk', 'rok', {'mode': '-'}),
 ('abrb', 'rok', {'mode': '-'}),
 ('abrb', 'sigh', {'mode': '-'}),
 ('abrb', 'comk', {'mode': '-'}),
 ('sigh', 'spo0a', {'mode': '+'}),
 ('siga', 'resd', {'mode': '+'}),
 ('siga', 'abrb', {'mode': '+'}),
 ('siga', 'sigd', {'mode': '+'}),
 ('siga', 'sigh', {'mode': '+'}),
 ('siga', 'med', {'mode': '+'}),
 ('siga', 'sinr', {'mode': '+'}),
 ('siga', 'comk', {'mode': '+'}),
 ('siga', 'phop', {'mode': '+'}),
 ('siga', 'spo0a', {'mode': '+'}),
 ('resd', 'phop', {'mode': '+'}),
 ('sigd', 'siga', {'mode': '+'}),
 ('med', 'comk', {'mode': '+'}),
 ('sinr', 'rok', {'mode': '-'}),
 ('phop', 'resd', {'mode': '-'}),
 ('spo0a', 'abrb', {'mode': '-'}),
 ('sigg', 'spovt', {'mode': '+'}),
 ('spovt', 'sigg', {'mode': '-'})]

In [23]:
print(nx.to_numpy_matrix(net, nodelist=gene_list, order={"C"}))

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]]
